In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from src.data.data_fetcher import get_all_features, get_raw_data
from src.features.feature_engineering import prepare_data
from src.features.preprocess_data import fetch_preprocessed_data, get_preprocessed_test_data

%pip install tensorflow
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

Note: you may need to restart the kernel to use updated packages.


In [27]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

In [28]:
X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
X_test_estimated_a_processed, X_test_estimated_b_processed, X_test_estimated_c_processed = get_preprocessed_test_data()

c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:131: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample('1H').mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:131: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.resample('1H').mean()
c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\src\features\feature_engineering.py:131: FutureWarning: The default value of numeric_o

In [29]:
(X_train_obs_combined.shape, X_val_obs_combined.shape, y_train_obs_combined.shape, y_val_obs_combined.shape, \
 X_train_est_combined.shape, X_val_est_combined.shape, y_train_est_combined.shape, y_val_est_combined.shape)

((2343, 45), (586, 45), (2343,), (586,), (440, 45), (110, 45), (440,), (110,))

In [30]:

# Scale the feature data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train_obs_combined)
y_train_scaled = scaler_Y.fit_transform(y_train_obs_combined.values.reshape(-1, 1))
X_val_scaled = scaler_X.transform(X_train_obs_combined)
y_val_scaled = scaler_Y.transform(y_train_obs_combined.values.reshape(-1, 1))

# Reshape input to be [samples, time steps, features]
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_val_reshaped = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))

# Construct the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val_scaled), verbose=2, shuffle=False)


Epoch 1/50
74/74 - 3s - loss: 0.0178 - val_loss: 0.0131 - 3s/epoch - 39ms/step
Epoch 2/50
74/74 - 0s - loss: 0.0104 - val_loss: 0.0091 - 224ms/epoch - 3ms/step
Epoch 3/50
74/74 - 0s - loss: 0.0093 - val_loss: 0.0088 - 226ms/epoch - 3ms/step
Epoch 4/50
74/74 - 0s - loss: 0.0088 - val_loss: 0.0082 - 215ms/epoch - 3ms/step
Epoch 5/50
74/74 - 0s - loss: 0.0094 - val_loss: 0.0101 - 221ms/epoch - 3ms/step
Epoch 6/50
74/74 - 0s - loss: 0.0091 - val_loss: 0.0111 - 213ms/epoch - 3ms/step
Epoch 7/50
74/74 - 0s - loss: 0.0087 - val_loss: 0.0088 - 227ms/epoch - 3ms/step
Epoch 8/50
74/74 - 0s - loss: 0.0084 - val_loss: 0.0079 - 228ms/epoch - 3ms/step
Epoch 9/50
74/74 - 0s - loss: 0.0082 - val_loss: 0.0076 - 223ms/epoch - 3ms/step
Epoch 10/50
74/74 - 0s - loss: 0.0082 - val_loss: 0.0076 - 228ms/epoch - 3ms/step
Epoch 11/50
74/74 - 0s - loss: 0.0079 - val_loss: 0.0078 - 220ms/epoch - 3ms/step
Epoch 12/50
74/74 - 0s - loss: 0.0082 - val_loss: 0.0076 - 262ms/epoch - 4ms/step
Epoch 13/50
74/74 - 0s - lo

In [31]:

# Make predictions on validation data
y_val_pred_scaled = model.predict(X_val_reshaped)

# Transform predictions back to original scale
y_val_pred = scaler_Y.inverse_transform(y_val_pred_scaled).flatten()

# Calculate Mean Absolute Error on validation data
# mae_val = mean_absolute_error(y_val_obs_combined, y_val_pred)
# mae_val

74/74 [==============================] - 1s 2ms/step


In [32]:
# Make predictions using the trained Linear Regression model on the processed test estimated data
from src.features.preprocess_data import get_final_prediction


y_pred_test_est_a = model.predict(X_test_estimated_a_processed)
y_pred_test_est_b = model.predict(X_test_estimated_b_processed)
y_pred_test_est_c = model.predict(X_test_estimated_c_processed)
# Combine the predictions from the three locations into a single dataframe
y_pred_test_est_a = scaler_Y.inverse_transform(y_pred_test_est_a).flatten()
y_pred_test_est_b = scaler_Y.inverse_transform(y_pred_test_est_b).flatten()
y_pred_test_est_c = scaler_Y.inverse_transform(y_pred_test_est_c).flatten()

y_pred_test_est_combined = get_final_prediction(y_pred_test_est_a, y_pred_test_est_b, y_pred_test_est_c)
y_pred_test_est_combined.head(20)


90/90 [==============================] - 0s 1ms/step


TypeError: in user code:

    File "c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\venv\lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\venv\lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\venv\lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\venv\lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlærin

## Step : Save model

In [ ]:

from src.models.saving import save_predictions


save_predictions(y_val_pred, 'lstm_no_feature_engineering')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices